In [1]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import torch

from mvn.utils.minimon import MiniMon
from mvn.utils.misc import flush_cache

from mvn.ipynb import get_args, get_config, build_labels
from train import setup_dataloaders, setup_experiment, build_env, do_train

In [3]:
flush_cache()

is_distributed = False
master = True
device = torch.device(0)
print('using dev {}'.format(device))
    
args = get_args()

# just to get a feeling of the dataset
# labels, mask, indices = build_labels(config.dataset.train.labels_path, 10000)
# labels, mask, indices = build_labels(config.dataset.train.labels_path, 10, allowed_subjects=['S9', 'S11'])

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |
|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |
|---------------------------------------------------------------------------|
| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |

In [72]:
config = get_config(args)
do_train(None, None, config, device, is_distributed, master)

cam2cam model:
                roto_extractor has       1886 params (~ 50.7) %
               trans_extractor has       1835 params (~ 49.3) %
total params:                 3721
cam2cam estimation => adding 3721 params to grad ...
  training dataset length: 12
  validation dataset length: 5
Experiment name: human36m_alg_AlgebraicTriangulationNet@11.05.2021-11:35:24
#[do_train]: cam2cam loss weights: geo: 0.0, trans: 0.0, proj: 1.0
#[do_train]: epoch    0 has started!
#[cam2cam]: I'm using GT 2D keypoints
#[cam2cam]: training batch iter 0 avg per sample loss: GEO ~ 2.384, TRANS ~ 69.445, POSE ~ 28275.584, ROTO ~ 3.006, TOTAL ~ 28275.584
#[cam2cam]: I'm using GT 2D keypoints
#[cam2cam]: training batch iter 1 avg per sample loss: GEO ~ 2.344, TRANS ~ 69.242, POSE ~ 27285.305, ROTO ~ 3.057, TOTAL ~ 27285.305
#[epoch]: training MPJPE relative to pelvis: 9906.657 mm
#[cam2cam]: I'm using GT 2D keypoints
#[epoch]: eval MPJPE relative to pelvis: 499.078 mm
#[do_train]: epoch time ~ 2.9" => 122

In [17]:
import numpy as np

E = np.float32([[-9.0557641e-01,4.2392653e-01,1.4752379e-02,-3.2391183e+02],
 [ 6.8628125e-02,1.8074372e-01,-9.8113298e-01,5.4177155e+02],
 [-4.1859469e-01,-8.8747847e-01,-1.9277054e-01,5.5065693e+03]])
R = E[:3, :3]
t = E[:3, 3]


def rotation_matrix_from_vectors(vec1, vec2):
    """ Find the rotation matrix that aligns vec1 to vec2
    :param vec1: A 3d "source" vector
    :param vec2: A 3d "destination" vector
    :return mat: A transform matrix (3x3) which when applied to vec1, aligns it with vec2.
    """
    a, b = (vec1 / np.linalg.norm(vec1)).reshape(3), (vec2 / np.linalg.norm(vec2)).reshape(3)
    v = np.cross(a, b)
    c = np.dot(a, b)
    s = np.linalg.norm(v)
    kmat = np.array([[0, -v[2], v[1]], [v[2], 0, -v[0]], [-v[1], v[0], 0]])
    rotation_matrix = np.eye(3) + kmat + kmat.dot(kmat) * ((1 - c) / (s ** 2))
    return rotation_matrix


z_axis = [0, 0, 1]
Rt = rotation_matrix_from_vectors(z_axis, [1, 5, 4])
Rt.dot(z_axis)

array([0.15430335, 0.77151675, 0.6172134 ])